<a href="https://colab.research.google.com/github/60hr00t/draft1/blob/main/sample_chatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets accelerate bitsandbytes sentencepiece fastapi uvicorn pyngrok nest_asyncio streamlit requests gTTS fuzzywuzzy python-Levenshtein streamlit-image-select streamlit-calendar pandas streamlit-mic-recorder openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.7/706.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.4 MB/s eta 0:00:00


In [2]:
# Detecting CUDA GPU with PyTorch
import torch, platform

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

In [3]:
# ==========================================================
# Upload a CSV with “prompt” and “response” columns in Colab
# and set `CSV_PATH` to the selected file.
# ==========================================================

import io, pandas as pd
from google.colab import files

print("Upload a CSV with columns: prompt,response")
uploaded = files.upload()
if uploaded:
    fname = list(uploaded.keys())[0]
    CSV_PATH = f"/content/{fname}"
    print("Using uploaded file:", CSV_PATH)


Upload a CSV with columns: prompt,response


Saving Vet.csv - Vet.csv to Vet.csv - Vet.csv
Using uploaded file: /content/Vet.csv - Vet.csv


In [4]:
# ==================================================
# Fuzzy match answers from CSV and render as plain
# text or option buttons
# ==================================================
from fuzzywuzzy import fuzz
df = pd.read_csv(CSV_PATH)
required = {"prompt", "response"}
assert required.issubset(df.columns), f"CSV must include: {required}. Found: {df.columns.tolist()}"
print(f"Loaded {len(df)} rows.")

# This function returns the best-matching response string for user_msg
# or None if the best score is below threshold.
def answer_from_csv(user_msg, df, threshold=90):
    best_score = 0
    best_response = None
    for _, row in df.iterrows():
        score = fuzz.partial_ratio(user_msg.lower(), row["prompt"].lower())
        if score > best_score:
            best_score = score
            best_response = row["response"]
    return best_response if best_score >= threshold else None
     # Check if response contains semicolons
    if ";" in response:
        options = [opt.strip() for opt in response.split(";")]
        st.markdown(f"**{prompt.capitalize()} Options:**")
    for i, option in enumerate(options):
            if st.button(option, key=f"{prompt}_{i}"):
                st.chat_message("user").markdown(option)
                reply = get_bot_response(option)  # Send to your backend
                st.chat_message("assistant").markdown(reply)
    else:
# If no semicolon, show as regular response
        st.markdown(f"**{prompt.capitalize()}:** {response}")




Loaded 2 rows.


In [5]:
# =====================================================================
# This snippet downloads and initializes Google’s FLAN-T5 (small) model
# and its tokenizer using the  Transformers library, then puts the model
# in evaluation mode for inference.
# =====================================================================
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [6]:
# ==========================================================================
# This section allows the FastAPI app exposes a chat endpoint that first looks
# up a reply in a CSV via answer_from_csv, falls back to a generative model
# if no strong match exists, synthesizes the reply to MP3 with gTTS, and serves
# the audio from a static /audio path.
# ==========================================================================
from fastapi import FastAPI
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
import uvicorn, nest_asyncio, threading, time, uuid, os
from pyngrok import conf, ngrok
from gtts import gTTS

app = FastAPI()
app.mount("/audio", StaticFiles(directory="/content", html=True), name="audio")

class Prompt(BaseModel):
    prompt: str

@app.post("/respond")
async def respond(request: Prompt):
    user_msg = request.prompt.strip()
    csv_answer = answer_from_csv(user_msg, df, threshold=90)
    filename = f"audio_{uuid.uuid4().hex[:8]}.mp3"
    filepath = f"/content/{filename}"

    if csv_answer:
        gTTS(text=csv_answer, lang='en').save(filepath)
        return {"response": csv_answer, "audio_path": f"/audio/{filename}"}

    prompt_text = f"You are a helpful cultural assistant. Answer clearly.\nUser: {user_msg}\nAssistant:"
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=120)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    gTTS(text=reply, lang='en').save(filepath)
    return {"response": reply, "audio_path": f"/audio/{filename}"}

@app.get("/audio/{filename}")
async def get_audio(filename: str):
    return FileResponse(path=f"/content/{filename}", media_type="audio/mpeg")


In [10]:
# =====================================================================
# This snippet runs the FastAPI app with Uvicorn in a background thread
# (using nest_asyncio), opens an ngrok tunnel with the auth token, then
# builds and prints the public /respond endpoint (FASTAPI_URL) and the
# base URL for serving audio (AUDIO_BASE_URL).
# ======================================================================
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

nest_asyncio.apply()
threading.Thread(target=run_api, daemon=True).start()
time.sleep(5)

conf.get_default().auth_token = "36Ig2CHEPLL7Ivf6J256Wgmd7i1_ThhuzTZLKDHhAEbosQCx"
public_url = ngrok.connect(8000)
FASTAPI_URL = f"{public_url.public_url}/respond"
AUDIO_BASE_URL = public_url.public_url
print("FASTAPI URL:", FASTAPI_URL)
print("AUDIO BASE:", AUDIO_BASE_URL)

INFO:     Started server process [189]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FASTAPI URL: https://unvibrated-tubercularly-darwin.ngrok-free.dev/respond
AUDIO BASE: https://unvibrated-tubercularly-darwin.ngrok-free.dev


In [11]:
import os
os.environ["SMTP_HOST"]="smtp.gmail.com"
os.environ["SMTP_PORT"]="587"
os.environ["SMTP_USER"]=""
os.environ["SMTP_PASS"]="your_app_password"
os.environ["FROM_EMAIL"]=""

In [12]:
from pathlib import Path

# Use config.py from existing env vars
Path("config.py").write_text(
    f'FASTAPI_URL = "{FASTAPI_URL}"\nAUDIO_BASE_URL = "{AUDIO_BASE_URL}"\n'
)

streamlit_code = """
import streamlit as st
import requests, base64, re, sys, os, html, json, ssl, smtplib, uuid
import pandas as pd
import streamlit.components.v1 as components
from email.message import EmailMessage
from datetime import datetime, timezone
from streamlit_image_select import image_select

# --- Mic and Speech to Text ---
try:
    from streamlit_mic_recorder import mic_recorder
    import whisper
    WHISPER_OK = True
except Exception:
    WHISPER_OK = False

# --- Whisper helpers ---
if WHISPER_OK:
    @st.cache_resource
    def _load_whisper_model():
        return whisper.load_model("tiny")

    def _stt_from_bytes(wav_bytes: bytes) -> str:
        import tempfile
        try:
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
                tmp.write(wav_bytes)
                tmp.flush()
                model = _load_whisper_model()
                result = model.transcribe(tmp.name, language="en")
                return (result.get("text") or "").strip()
        except Exception as e:
            st.error(f"Speech-to-text failed: {e}")
            return ""
else:
    def _stt_from_bytes(_):
        return ""

# ---- Load API URLs ----
if "" not in sys.path:
    sys.path.insert(0, "")

try:
    from config import FASTAPI_URL, AUDIO_BASE_URL
except Exception as e:
    FASTAPI_URL = ""
    AUDIO_BASE_URL = ""
    st.warning(f"Could not import FASTAPI_URL/AUDIO_BASE_URL from config.py. Re-run your ngrok cell. Details: {e}")

# Initialize session state
if "greeted" not in st.session_state:
    st.session_state.greeted = False
if "user_name" not in st.session_state:
    st.session_state.user_name = ""
if "star_rating" not in st.session_state:
    st.session_state["star_rating"] = 0
if "star_feedback" not in st.session_state:
    st.session_state["star_feedback"] = ""
if "star_submitted" not in st.session_state:
    st.session_state["star_submitted"] = False

# Display popup to capture user name
if st.session_state.user_name == "":
    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        st.markdown('''
            <div style="
                background-color: #009B3A;
                padding: 30px;
                border-radius: 15px;
                box-shadow: 0 4px 12px rgba(0,0,0,0.2);
                text-align: center;
                color: white;">
                <h3>🐾 Welcome to Jamaica Vet Finder</h3>
                <p>What's your name?</p>
            </div>
        ''', unsafe_allow_html=True)
        name = st.text_input(" ", key="name_input")

        if name:
            st.session_state["user_name"] = name
            st.rerun()
    st.stop()

if "avatar_chosen" not in st.session_state:
    st.session_state.avatar_chosen = False
if "avatar" not in st.session_state:
    st.session_state.avatar = None

st.set_page_config(page_title="Jamaica Vet Finder", page_icon="🐾")

# --- Jamaican themed sidebar ---
st.markdown('''
    <style>
    [data-testid="stSidebar"] {
        background: linear-gradient(135deg, #009B3A 0%, #FED100 100%);
        color: white;
        border-right: 2px solid #000000;
    }
    [data-testid="stSidebar"] .css-1cpxqw2 { color: black; }
    button[kind="primary"] {
        background-color: #009B3A !important;
        color: white !important;
        border-radius: 8px !important;
        font-weight: bold;
        border: none !important;
    }
    label, .stCheckbox { color: black !important; }
    div[data-testid="baseButton-secondary"] button {
        font-size: 26px !important;
        line-height: 26px !important;
        padding: 6px 10px !important;
    }
    </style>
''', unsafe_allow_html=True)

# Initialize chat session
if "avatar" not in st.session_state:
    st.session_state.avatar = None
if "avatar_chosen" not in st.session_state:
    st.session_state.avatar_chosen = False
if "initial_greeting" not in st.session_state:
    st.session_state.initial_greeting = None
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []
if "chat_ended" not in st.session_state:
    st.session_state["chat_ended"] = False
if "show_vet_map" not in st.session_state:
    st.session_state.show_vet_map = False
if "open_email_panel" not in st.session_state:
    st.session_state.open_email_panel = False

# =========================
# FastAPI section
# =========================
def get_bot_response(msg):
    api_ready_local = bool(FASTAPI_URL and AUDIO_BASE_URL)
    if not api_ready_local:
        return "API not configured. Re-run ngrok to write config.py.", None
    try:
        payload = {"prompt": msg}
        res = requests.post(FASTAPI_URL, json=payload, timeout=30)
        if res.status_code != 200:
            detail = ""
            try:
                j = res.json(); detail = j.get("detail") or j
            except Exception:
                detail = res.text
            return f"Server error {res.status_code}: {detail}", None
        try:
            data = res.json()
        except Exception:
            return f"Server returned non-JSON:\\n{res.text}", None
        return data.get("response", "No response."), data.get("audio_path")
    except Exception as e:
        return f"Error contacting server: {e}", None

# =========================
# CSV and Vet Data
# =========================
def _find_vet_data_csv():
    candidates = []
    for d in ["/content", "."]:
        try:
            for fname in os.listdir(d):
                if fname.lower().endswith(".csv"):
                    path = os.path.join(d, fname)
                    try:
                        mtime = os.path.getmtime(path)
                        candidates.append((mtime, path))
                    except Exception:
                        pass
        except Exception:
            pass
    candidates.sort(reverse=True)
    for _, path in candidates:
        try:
            df = pd.read_csv(path)
            cols = {c.strip().lower() for c in df.columns}
            if {"prompt", "response"}.issubset(cols):
                return path
        except Exception:
            continue
    return None

def _parse_vet_locations(df: pd.DataFrame):
    vets = []
    df = df.rename(columns={c: c.strip().lower() for c in df.columns})
    for _, row in df.iterrows():
        p = str(row.get("prompt", "")).strip()
        r = str(row.get("response", "")).strip()
        pl = p.lower()

        if any(k in pl for k in ["vet", "veterinary", "clinic", "animal hospital", "pet care"]):
            items = [x.strip() for x in re.split(r";|\\n", r) if x.strip()]
            for it in items:
                vet_info = {"name": "", "address": "", "phone": "", "parish": "", "services": ""}

                parts = it.split("|")
                if len(parts) >= 2:
                    vet_info["name"] = parts[0].strip()
                    vet_info["address"] = parts[1].strip()
                    if len(parts) >= 3:
                        vet_info["phone"] = parts[2].strip()
                    if len(parts) >= 4:
                        vet_info["parish"] = parts[3].strip()
                    vets.append(vet_info)

    return vets

def load_vets_from_csv():
    path = _find_vet_data_csv()
    if not path: return [], None
    try:
        df = pd.read_csv(path)
        vets = _parse_vet_locations(df)
        return vets, path
    except Exception as e:
        st.warning(f"Failed to read vet data from CSV: {e}")
        return [], path

# =========================
# Email functionality
# =========================
def _get_smtp_config():
    host = st.secrets.get("smtp_host", os.environ.get("SMTP_HOST"))
    port = st.secrets.get("smtp_port", os.environ.get("SMTP_PORT", "587"))
    user = st.secrets.get("smtp_user", os.environ.get("SMTP_USER"))
    pwd  = st.secrets.get("smtp_password", os.environ.get("SMTP_PASS"))
    from_email = st.secrets.get("from_email", os.environ.get("FROM_EMAIL", user))
    try: port = int(port)
    except Exception: port = 587
    return host, port, user, pwd, from_email

def send_vet_info_email(to_email: str, vet_info: dict):
    host, port, user, pwd, from_email = _get_smtp_config()
    if not all([host, port, user, pwd, from_email]):
        return False, "SMTP not configured."

    msg = EmailMessage()
    msg["Subject"] = f"Vet Information: {vet_info.get('name', 'Veterinary Clinic')}"
    msg["From"] = from_email
    msg["To"] = to_email

    # FIXED — NO TRIPLE QUOTES ANYWHERE
    body = (
        "Hello,\n\n"
        "Here’s the veterinary clinic information you requested:\n\n"
        f"Name: {vet_info.get('name', 'N/A')}\n"
        f"Address: {vet_info.get('address', 'N/A')}\n"
        f"Phone: {vet_info.get('phone', 'N/A')}\n"
        f"Parish: {vet_info.get('parish', 'N/A')}\n"
        f"Services: {vet_info.get('services', 'N/A')}\n\n"
        "— Jamaica Vet Finder\n"
    )
    msg.set_content(body)

    try:
        context = ssl.create_default_context()
        with smtplib.SMTP(host, port) as server:
            server.starttls(context=context)
            server.login(user, pwd)
            server.send_message(msg)
        return True, "Email sent successfully!"
    except Exception as e:
        return False, f"Failed to send: {e}"

# =========================
# User Interface
# =========================
avatars = [
    "https://i.imgur.com/bOxErmA.png",
    "https://i.imgur.com/UABUwk7.png",
    "https://i.imgur.com/IKOW70q.png",
    "https://i.imgur.com/PNpZF0X.png",
]

st.title("Jamaica Vet Finder 🐾🇯🇲")
st.markdown("Find veterinary care for your pets across Jamaica")

st.button("🗺️ View Vet Locations", on_click=lambda: st.session_state.update(show_vet_map=True))

if not st.session_state.avatar_chosen:
    st.markdown("### 👤 Choose your avatar to start")
    selected_avatar = image_select("Pick your avatar:", avatars, use_container_width=True)

    if selected_avatar:
        st.session_state.avatar = selected_avatar
        st.image(selected_avatar, width=100)

    if st.button("Next"):
        if st.session_state.avatar:
            st.session_state.avatar_chosen = True
            st.rerun()
        else:
            st.warning("Please pick your avatar before proceeding")

chat_placeholder = st.container()

api_ready = bool(FASTAPI_URL and AUDIO_BASE_URL)
if not api_ready:
    st.error("FASTAPI_URL / AUDIO_BASE_URL not set. Re-run your ngrok cell.")

chat_placeholder = st.container()

if st.session_state.avatar_chosen:
    if st.session_state.initial_greeting is None:
        greeting = (
            "🐾 Welcome to Jamaica Vet Finder! "
            "I'm here to help you find the best veterinary care for your pets across Jamaica. "
            "Whether you need a routine checkup, emergency care, or specialist services, "
            "I can help you locate vets in all 14 parishes. "
            "Ask me about vets near you, emergency services, or specific pet care needs!"
        )
        st.session_state.chat_history.append(("VetBot", greeting))
        st.session_state.initial_greeting = greeting

    # Sidebar
    if st.session_state.avatar_chosen:
        st.sidebar.title("Jamaica Vet Finder 🐾")
        with st.sidebar:
            if st.session_state.get("user_name"):
                st.markdown(f"### 👋 Hello {st.session_state['user_name']}!")
            else:
                st.markdown("### 👋 Welcome!")

        st.sidebar.title("🧹 Chat Controls")
        st.sidebar.markdown("Want to start fresh?")
        confirm_clear = st.sidebar.checkbox("Yes, I'm sure")

        if st.sidebar.button("Clear Chat History", disabled=not confirm_clear):
            st.session_state.chat_history = []
            st.session_state.initial_greeting = None
            st.session_state.chat_history.append(("VetBot","Chat cleared! Let's start fresh."))
            st.sidebar.success("Chat history cleared!")

        st.sidebar.title("💬 Chat History")
        for i, msg in enumerate(st.session_state.chat_history):
            if isinstance(msg, tuple) and len(msg) == 2:
                sender, content = msg
                if "Server error" not in content:
                    st.sidebar.markdown(f"**{sender}:** {content[:50]}...")

    # Input row with mic + send
    with st.container():
        col1, col2 = st.columns([8, 2])
        with col1:
            ic1, ic2 = st.columns([12, 1])
            with ic1:
                user_input = st.text_input("Ask about vets in Jamaica:", key="user_input", label_visibility="collapsed")
            with ic2:
                if WHISPER_OK:
                    voice = mic_recorder(start_prompt="🎤", stop_prompt="⏹️", just_once=True, format="wav", key="vet_mic")
                else:
                    st.button("🎤", disabled=True)
        with col2:
            send_clicked = st.button("Send", use_container_width=True, disabled=not api_ready)

# End conversation and rating
rating_labels = {1:"Poor", 2:"Fair", 3:"Good", 4:"Very Good", 5:"Excellent"}

def _set_star_rating(v: int):
    st.session_state["star_rating"] = int(v)

if st.button("End Conversation"):
    st.session_state["chat_ended"] = True

if st.session_state.get("chat_ended"):
    st.markdown("### 🐾 Thank You!")
    st.success("Thanks for using Jamaica Vet Finder!")

    card = st.container()
    with card:
        st.markdown("#### ⭐ Rate your experience")
        star_cols = st.columns(5)
        for i in range(1, 6):
            with star_cols[i-1]:
                star_char = "★" if st.session_state["star_rating"] >= i else "☆"
                st.button(star_char, key=f"star_{i}", help=f"{i} - {rating_labels[i]}",
                          on_click=lambda v=i: _set_star_rating(v))

        if st.session_state["star_rating"] > 0:
            sel = st.session_state["star_rating"]
            st.markdown(f"**You selected {sel} star{'s' if sel>1 else ''}: {rating_labels[sel]}**")
            st.session_state["star_feedback"] = st.text_area("Any feedback?", value=st.session_state.get("star_feedback",""))

            if st.button("Submit rating", key="submit_rating"):
                st.session_state["star_submitted"] = True
                st.success("Thanks for your rating! 🌟")

# Handle voice input
if WHISPER_OK and 'voice' in locals() and isinstance(voice, dict) and voice.get("bytes"):
    transcript = _stt_from_bytes(voice["bytes"])
    if transcript:
        st.session_state.chat_history.append(("User", transcript))
        reply, audio_path = get_bot_response(transcript)
        st.session_state.chat_history.append(("VetBot", reply))
        if audio_path:
            try:
                full_audio_url = f"{AUDIO_BASE_URL}{audio_path}"
                audio_bytes = requests.get(full_audio_url, timeout=30).content
                st.audio(audio_bytes, format="audio/mp3")
            except Exception as e:
                st.warning(f"Failed to load audio: {e}")

# Handle typed input
if 'user_input' in st.session_state and send_clicked and st.session_state.user_input:
    val = st.session_state.user_input
    st.session_state.chat_history.append(("User", val))

    reply, audio_path = get_bot_response(val)
    st.session_state.chat_history.append(("VetBot", reply))
    if audio_path:
        try:
            full_audio_url = f"{AUDIO_BASE_URL}{audio_path}"
            audio_bytes = requests.get(full_audio_url, timeout=30).content
            st.audio(audio_bytes, format="audio/mp3")
        except Exception as e:
            st.warning(f"Failed to load audio: {e}")

# Chat bubble CSS
chat_bubble_css = '''
<style>
.block-container { background: linear-gradient(to bottom, #ffffff 60%, #e8f5e9 100%); padding: 20px; }
@keyframes fadeInUser { from { opacity: 0; transform: translateY(6px); } to { opacity: 1; transform: translateY(0); } }
@keyframes fadeInBot  { 0% { opacity: 0; transform: translateY(6px); } 40% { opacity: 0; transform: translateY(6px); } 100% { opacity: 1; transform: translateY(0); } }
.chat-container { max-width: 900px; margin: 6px auto 80px auto; padding: 4px 8px; }
.bubble-row { width: 100%; display: flex; margin: 6px 0; }
.bubble-row.newest-user { animation: fadeInUser 0.4s ease-out; }
.bubble-row.newest-bot  { animation: fadeInBot 1.2s ease-out; }
.bubble { padding: 10px 14px; border-radius: 18px; max-width: 75%; word-wrap: break-word; box-shadow: 0 2px 8px rgba(0,0,0,0.08); }
.bot { background: #FED100; color: #111; border-top-left-radius: 6px; }
.user { background: #009B3A; color: white; border-top-right-radius: 6px; margin-left: auto; }
.sender-label { font-size: 12px; opacity: 0.8; margin-bottom: 4px; }
.msg-text { line-height: 1.4; white-space: pre-wrap; }
</style>
'''
st.markdown(chat_bubble_css, unsafe_allow_html=True)

def html_escape(s: str): return html.escape(s or "")

with chat_placeholder:
    st.markdown('<div class="chat-container">', unsafe_allow_html=True)
    for i, (sender, raw_msg) in enumerate(st.session_state.chat_history):
        is_user = sender == "User"
        role_class = "user" if is_user else "bot"
        align_style = "justify-content:flex-end;" if is_user else "justify-content:flex-start;"

        bubble_html = f'<div class="bubble-row" style="{align_style}"><div class="bubble {role_class}">'
        bubble_html += f'<div class="sender-label">{html_escape(sender)}</div>'
        bubble_html += f'<div class="msg-text">{html_escape(raw_msg)}</div>'
        bubble_html += '</div></div>'

        st.markdown(bubble_html, unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)
"""

Path("vet_finder_app.py").write_text(streamlit_code)
print("Saved vet_finder_app.py")


Saved vet_finder_app.py


In [14]:
 # =================================================================
# This section terminates any process on port 8501, launches the
# Streamlit app headlessly on that port with logs to /content/streamlit_log.txt
# in the background, waits 10 seconds, then opens an ngrok tunnel to 8501 and
# prints the public URL.
# =================================================================
!fuser -k 8501/tcp || true
!streamlit run chatbot_streamlit_app.py --server.headless true --server.port 8501 > /content/streamlit_log.txt 2>&1 &
time.sleep(10)
streamlit_url = ngrok.connect(8501).public_url
print("Streamlit available at:", streamlit_url)

Streamlit available at: https://unvibrated-tubercularly-darwin.ngrok-free.dev
